In [18]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd /kaggle/working/segment-anything-2
%pip install -e .

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 974, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 974 (delta 17), reused 21 (delta 7), pack-reused 936 (from 1)
Receiving objects: 100% (974/974), 128.94 MiB | 38.65 MiB/s, done.
Resolving deltas: 100% (334/334), done.
/kaggle/working/segment-anything-2
Obtaining file:///kaggle/working/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=SAM_2-1.0-0.editable-py3-none-any.whl size=13393 sha256=3b76036f33b046260f84db8df889d228e27044c634e517c9f7f2cbd255e4ae93
  Stored in directory: /tmp/pip-ephem-wheel-cache-l6g2x6lm/wheels/1b/30/72/94667a625e111d514f83ac75152c8957c764a2c56d72e883a6
Successfull

In [19]:
import sys
sys.path.append("/kaggle/input/segment-anything-2/pytorch/sam2-hiera-base-plus/1")

!pip install modelbit

import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

> > 

In [20]:
def save_image(image, filename):
    cv2.imwrite(os.path.join(training_images_dir, filename), image)


In [21]:
# Function to extract objects from the provided images
def extract_objects_from_images(image_paths):
    extracted_objects = []
    
    for img_path in image_paths:
        # Load image
        image = np.array(Image.open(img_path).convert("RGB"))
        
        # Generate masks
        masks = mask_generator.generate(image)
        
        for mask in masks:
            object_image = extract_object(image, mask['segmentation'])
            extracted_objects.append(object_image)
    
    return extracted_objects

In [22]:
def extract_bounding_box(mask_list):
    """
    Extract bounding box coordinates from a list of masks.
    Returns list of (xmin, ymin, xmax, ymax) for each mask.
    """
    bounding_boxes = []
    for mask in mask_list:
        binary_mask = mask.astype(np.uint8) * 255
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:  # Check if contour exists
            x, y, w, h = cv2.boundingRect(contours[0])
            bounding_boxes.append((x, y, x + w, y + h))
    return bounding_boxes

In [23]:
def track_object_with_bounding_box(object_image, target_image):
    """
    Track the object in the target image by matching features and drawing a bounding box.
    
    Parameters:
    - object_image: Extracted object from the first image.
    - target_image: The second image where the object is searched.

    Returns:
    - Target image with the bounding box drawn.
    """
    # Convert both images to grayscale
    if object_image.size == 0 or target_image.size == 0:
        print("Error: Empty image or object crop detected.")
        return target_image
    gray_object = cv2.cvtColor(object_image, cv2.COLOR_RGB2GRAY)
    gray_target = cv2.cvtColor(target_image, cv2.COLOR_RGB2GRAY)

    # Initialize ORB detector for feature matching
    orb = cv2.ORB_create(5000)  # Extract up to 5000 keypoints
    kp1, des1 = orb.detectAndCompute(gray_object, None)
    kp2, des2 = orb.detectAndCompute(gray_target, None)

    # Match features using BFMatcher with Hamming distance
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) < 4:
        print("Not enough matches found!")
        return target_image

    # Extract keypoints for homography
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute homography matrix
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Get object corners and transform them to the target image
    h, w = gray_object.shape
    obj_corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
    transformed_corners = cv2.perspectiveTransform(obj_corners, H)

    # Extract the bounding box coordinates from transformed corners
    xmin = int(min(transformed_corners[:, 0, 0]))
    ymin = int(min(transformed_corners[:, 0, 1]))
    xmax = int(max(transformed_corners[:, 0, 0]))
    ymax = int(max(transformed_corners[:, 0, 1]))

    # Draw the bounding box on the target image
    result_image = target_image.copy()
    cv2.rectangle(result_image, (xmin, ymin), (xmax, ymax), (0, 255, 0), 3)

    return result_image

In [ ]:
def track_multiple_objects(first_image_list, first_mask_list, second_image):
    """
    Track multiple objects in the second image by matching features and drawing bounding boxes.
    
    Parameters:
    - first_image_list: List of object images extracted from the first set of images.
    - first_mask_list: List of masks for the corresponding object images.
    - second_image: The second image where the objects will be searched.

    Returns:
    - Second image with bounding boxes drawn for all detected objects.
    """
    result_image = second_image.copy()  # Copy the second image for drawing bounding boxes
    
    '''# Loop through each object and its corresponding mask
    for i in range(len(first_image_list)):
        print(f"Tracking object {i + 1}...")

        # Extract the bounding box for the current object'''
    object_bbox = extract_bounding_box([first_mask_list[i]])[0]

        # Track the object and get the result image with the bounding box drawn
    tracked_image = track_object_with_bounding_box(first_image_list[i], result_image)
        
   ''' # Display the intermediate results (optional, can be commented out)
    plt.figure(figsize=(8, 8))
    plt.imshow(tracked_image)
    plt.axis('off')
    plt.show()'''
        
    # Update the result image
    result_image = tracked_image
    
    return result_image

In [25]:
def crop_object_with_mask(image, mask):
    """
    Crops the object tightly using the given mask.
    
    Parameters:
    - image (numpy array): The original image.
    - mask (numpy array): The mask indicating the object area.

    Returns:
    - Cropped object image (numpy array).
    """
    # Ensure mask is binary
    binary_mask = mask.astype(np.uint8) * 255

    # Find contours of the mask to get the tight bounding box
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    x, y, w, h = cv2.boundingRect(contours[0])

    # Crop the image using the bounding box
    cropped_image = image[y:y + h, x:x + w]

    return cropped_image

In [26]:

import os

def search_jpg_files(directory):
    """
    Search for all .jpg files in the given directory and its subdirectories.

    Parameters:
    - directory: Path to the directory to search.

    Returns:
    - A list of paths to the found .jpg files.
    """
    jpg_files = []

    # Walk through all subdirectories and files in the given directory
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.jpg'):  # Check for .jpg extension (case insensitive)
                jpg_files.append(os.path.join(root, file))

    return jpg_files

In [27]:
def calculate_iou(pred_box, gt_box):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - pred_box: Tuple (xmin, ymin, xmax, ymax) for predicted bounding box.
    - gt_box: Tuple (xmin, ymin, xmax, ymax) for ground truth bounding box.

    Returns:
    - IoU value (float) between 0 and 1.
    """
    # Calculate intersection coordinates
    x1 = max(pred_box[0], gt_box[0])
    y1 = max(pred_box[1], gt_box[1])
    x2 = min(pred_box[2], gt_box[2])
    y2 = min(pred_box[3], gt_box[3])

    # Calculate intersection area
    intersection_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Calculate areas of predicted and ground truth boxes
    pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
    gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])

    # Calculate union area
    union_area = pred_area + gt_area - intersection_area

    # Avoid division by zero
    if union_area == 0:
        return 0.0

    # Calculate IoU
    return intersection_area / union_area

In [28]:
def precision_recall(pred_boxes, gt_boxes, iou_threshold=0.5):
    """
    Calculate precision and recall for predicted and ground truth bounding boxes.

    Parameters:
    - pred_boxes: List of predicted bounding boxes [(xmin, ymin, xmax, ymax), ...].
    - gt_boxes: List of ground truth bounding boxes [(xmin, ymin, xmax, ymax), ...].
    - iou_threshold: IoU threshold to consider a prediction as a true positive.

    Returns:
    - Precision and recall values (float).
    """
    tp, fp, fn = 0, 0, 0  # Initialize counters
    matched = [False] * len(gt_boxes)  # Track matched ground truth boxes

    # Iterate over all predicted boxes
    for pred_box in pred_boxes:
        found_match = False

        # Check against all ground truth boxes
        for i, gt_box in enumerate(gt_boxes):
            if not matched[i] and calculate_iou(pred_box, gt_box) >= iou_threshold:
                tp += 1  # True Positive
                matched[i] = True  # Mark ground truth as matched
                found_match = True
                break

        if not found_match:
            fp += 1  # False Positive

    # Count unmatched ground truth boxes as false negatives
    fn = matched.count(False)

    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return precision, recall

In [29]:
def calculate_map(pred_boxes_list, gt_boxes_list, iou_threshold=0.5):
    """
    Calculate Mean Average Precision (mAP) for multiple predictions and ground truths.

    Parameters:
    - pred_boxes_list: List of predicted boxes for multiple images.
    - gt_boxes_list: List of ground truth boxes for corresponding images.
    - iou_threshold: IoU threshold for calculating true positives.

    Returns:
    - mAP value (float).
    """
    precisions = []
    recalls = []

    # Calculate precision and recall for each image
    for pred_boxes, gt_boxes in zip(pred_boxes_list, gt_boxes_list):
        precision, recall = precision_recall(pred_boxes, gt_boxes, iou_threshold)
        precisions.append(precision)
        recalls.append(recall)

    # Calculate average precision (AP) for the given IoU threshold
    ap = sum(precisions) / len(precisions) if precisions else 0

    return ap


In [ ]:
# Create a directory to store training images
training_images_dir = "/kaggle/input/cmu-dataset-images/CMU10_3D/modelimages"
os.makedirs(training_images_dir, exist_ok=True )

# Load and initialize SAM2 model
checkpoint_path = "/kaggle/input/segment-anything-2/pytorch/sam2-hiera-base-plus/1/sam2_hiera_base_plus.pt"
model_config = "sam2_hiera_b+.yaml"
sam2_model = build_sam2(model_config, checkpoint_path, device='cuda', apply_postprocessing=False)
mask_generator = SAM2AutomaticMaskGenerator(sam2_model)

# Load images from the directory and extract objects
image_files = os.listdir(training_images_dir)
image_paths = [os.path.join(training_images_dir, img) for img in image_files if img.endswith(('.jpg', '.png'))]

# Extract objects from the images in the training directory
extracted_objects = extract_objects_from_images(image_paths)
#print("Extracted_Objects",extracted_objects)
# Display extracted objects
'''for obj in extracted_objects:
    plt.figure(figsize=(8, 8))
    plt.imshow(obj)
    plt.axis('off')
    plt.show()'''

import numpy as np
# Load the first image and mask
path = "/kaggle/input/cmu-dataset-images/CMU10_3D/modelimages/"
objects = ['can_chowder/can_chowder_01.jpg','can_chowder/can_chowder_02.jpg','can_chowder/can_chowder_03.jpg','can_chowder/can_chowder_04.jpg','can_chowder/can_chowder_05.jpg',
          'can_soymilk/can_soymilk_01.jpg','can_soymilk/can_soymilk_02.jpg','can_soymilk/can_soymilk_03.jpg','can_soymilk/can_soymilk_04.jpg','can_soymilk/can_soymilk_05.jpg',
          'can_tomatosoup/can_tomatosoup_01.jpg','can_tomatosoup/can_tomatosoup_02.jpg','can_tomatosoup/can_tomatosoup_03.jpg','can_tomatosoup/can_tomatosoup_04.jpg','can_tomatosoup/can_tomatosoup_05.jpg',
          'carton_oj/carton_oj_01.jpg','carton_oj/carton_oj_02.jpg','carton_oj/carton_oj_03.jpg','carton_oj/carton_oj_04.jpg','carton_oj/carton_oj_05.jpg',
          'carton_soymilk/carton_soymilk_01.jpg','carton_soymilk/carton_soymilk_02.jpg','carton_soymilk/carton_soymilk_03.jpg','carton_soymilk/carton_soymilk_04.jpg','carton_soymilk/carton_soymilk_05.jpg',
          'diet_coke/diet_coke_01.jpg','diet_coke/diet_coke_02.jpg','diet_coke/diet_coke_03.jpg','diet_coke/diet_coke_04.jpg','diet_coke/diet_coke_05.jpg',
          'hc_potroastsoup/hc_potroastsoup_01.jpg','hc_potroastsoup/hc_potroastsoup_02.jpg','hc_potroastsoup/hc_potroastsoup_03.jpg','hc_potroastsoup/hc_potroastsoup_04.jpg','hc_potroastsoup/hc_potroastsoup_05.jpg',
          'juicebox/juicebox_01.jpg','juicebox/juicebox_02.jpg','juicebox/juicebox_03.jpg','juicebox/juicebox_04.jpg','juicebox/juicebox_05.jpg',
           'rice_tuscan/rice_tuscan_01.jpg','rice_tuscan/rice_tuscan_02.jpg','rice_tuscan/rice_tuscan_03.jpg','rice_tuscan/rice_tuscan_04.jpg','rice_tuscan/rice_tuscan_05.jpg',
           'ricepilaf/ricepilaf_01.jpg','ricepilaf/ricepilaf_02.jpg','ricepilaf/ricepilaf_03.jpg','ricepilaf/ricepilaf_04.jpg','ricepilaf/ricepilaf_05.jpg']
object_png = [obj.replace('.jpg', '.png') for obj in objects]
first_image_list = []
first_mask_list = []
object_cropped = []

gt_boxes_list = []
pred_boxes_list = []
for i in objects:
    #print(path+i)
    first_image_list.append(np.array(Image.open(path+i).convert("RGB")))
for i in object_png:
    first_mask_list.append(np.array(Image.open(path+i)))

# Crop the object tightly from the first image
for i in range(len(objects)):
    object_cropped = crop_object_with_mask(first_image_list[i], first_mask_list[i])
    gt_boxes_list.append(object_cropped)

        
#Example usage:
directory_path = '/kaggle/input/cmu-dataset-images/CMU10_3D/data_2D'  # Replace with your target directory path
jpg_files = search_jpg_files(directory_path)
            
# Load the second image where the object will be searched
search_path = "/kaggle/input/cmu-dataset-images/CMU10_3D/data_2D"

for jpg in range(len(jpg_files)//4):
    second_image = np.array(Image.open(jpg_files[jpg]).convert("RGB"))
    #print("Second_image",second_image)
    # Extract the bounding box from the first image's mask
    object_bbox = extract_bounding_box(first_mask_list)
    pred_boxes_list.append(object_bbox)  # Assuming bbox is already in format [x_min, y_min, x_max, y_max]
    # Track and highlight the object in the second image
    final_tracked_image = track_multiple_objects(first_image_list, first_mask_list, second_image)
    tracked_image = track_object_with_bounding_box(first_image_list[i], second_image)
    filename = jpg_files[jpg]        
    save_image(tracked_image, filename)
    '''plt.figure(figsize=(10, 10))
    plt.imshow(final_tracked_image)
    plt.axis('off')
    plt.show()'''

In [ ]:
#calculate_map(pred_boxes_list, gt_boxes_list, iou_threshold=0.5)